## 서론

혐오발언은 혐오범죄로 이어질 수 있기 때문에, 혐오발언을 하는 sns 사용자를 식별해내는 것은 잠재적인 범죄를 예방할 수 있다는 점에서 중요하다. 공격적인 표현이 많은, 소셜 미디어의 특성 상 특정 집단에 대한 혐오를 드러내는 혐오발언을 단순히 비속어를 포함하는 공격적인 발언과 구분하기가 어려울 수밖에 없다.

어떤 트윗이 혐오발언과 공격적인 발언, 그리고 중립적인 표현인지 분류하기 위해 Davidson(2017)은 “Automated Hate Speech Detection and the Problem of Offensive Language”에서 트윗들을 소문자화하고 어간을 추출한 후, n-gram 언어모형과 Tf-Idf을 특성값으로 하는 로지스틱 회귀분석 분류기를 사용했다. 본고에서는 이와 함께 Waseem&Hovy(2016)의 “Hateful Symbols or Hateful People? Predictive Features for Hate Speech Detection on Twitter”에서 트윗의 길이를 고려한 것을 참고하여 데이터를 전처리하고 특성값을 설정하였다. 분류기는 로지스틱 회귀분석 기법을 사용하였고 cross validation을 위해 GridSearchCV를 이용하였다. 


먼저 여러 모듈을 import하고, 필요한 것들을 다운로드 받았다. 또한, 악성 트윗 분류기 개발을 위해 필요한 "train_data.csv" 파일을 읽어들였다.

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data = pd.read_csv("train_data.csv",index_col=0)

## 데이터 전처리

데이터 전처리를 위해 import re, import nltk 등을 사용하였다.

데이터 전처리(preprocess(text))를 해주는 preprocess(text) 함수는 아래와 같은 항목들을 통해 구성되고 정의되었다.

<br/>

**1. 조금 더 깔끔한 문장 만들기 (부정문, 미래형, 'm, 're의 줄임말 풀어 써주기)**
- sentence = sentence.replace('n\\'t', ' not')
- sentence = sentence.replace('\\'ll', ' will')
- sentence = sentence.replace('\\'m', ' am')
- sentence = sentence.replace('\\'re', ' are')
> 이를 통해 조금 더 가독성 있는 문장이 되었다. 예를 들어 shouln't는 should not으로, don't는 do not으로, I'll은 I will, I'm은 I am, You're은 You are로 변화되었다.

**2. 공백 처리**
- space_pattern = '\s+'
- result = re.sub(space_pattern, ' ', sentence)
> 공백을 나타내는 정규표현식을 쓴 후 re.sub를 사용하여 불필요하게 많이 나오는 공백을 띄어쓰기로 바꾸어 주었다.

**3. url 처리**
- url = ('http\S+')
- result = re.sub(url, '', result)
> url(https://~ 혹은 http://~)을 나타내는 정규표현식을 쓴 후 re.sub를 사용하여 트윗에 포함되어 있는 url들을 없애주었다.

**4. 문장 앞쪽에 나오는 rt 멘션 처리**
- rt_mention = '([Rr][Tt]\s@[A-Za-z0-9_]+)'
- result = re.sub(rt_mention, '', result)
> 많은 트윗들이 문장 앞쪽에 RT @아이디의 형식을 갖고 있음을 볼 수 있다. 이는 '리트윗'의 의미를 지니고 있는데, 본 보고서에서는 이것이 악성 트윗을 분류하는 데 영향이 별로 없을 것이라고 판단하여 이를 나타내는 정규표현식을 쓴 후 re.sub를 통해 트윗에 포함되어 있는 리트윗 형식을 제거하였다.

**5. 문장 중간에 나오는 멘션 처리**
- middle_mention = '(@[A-Za-z0-9_]+)'
- result = re.sub(middle_mention, '', result)
> 문장 맨 앞에 RT @아이디의 형식을 지닌 것뿐만 아니라, 문장 중간중간에 @아이디의 문자열을 갖고 있는 트윗도 많다. 이는 앞서 지정한 rt_mention과는 다르게 'RT'를 앞에 포함하고 있지 않은데, 따라서 middle_mention이라고 하여 따로 구분해주었다. 이것 역시 악성 트윗을 분류하는 데 영향이 별로 없다고 판단하여 이를 나타내는 정규표현식을 쓴 후, re.sub를 통해 제거하였다.

**6. 단어가 아닌 숫자나 기호 등 처리 (빈칸이나 몇몇 문장부호 제외)**
- not_words = "[^A-Za-z.,!?\\'\\"\s]*"
- result = re.sub(not_words, '', result)
> 트윗을 보면 숫자나 여러 불필요한 기호들이 중간에 삽입되어 글이 매우 지저분함을 볼 수 있다. 그래서 글을 더 깔끔하게 만들기 위해서 단어가 아닌 숫자나 기호인 경우를 정규표현식으로 지정하여 이를 re.sub를 통해 제거하였다.
- 문장부호는 아래 7번에서 따로 처리한다.

**7. 문장부호 처리**
- punctuations = '[!\\?.,:;\\"\\']'
- result = re.sub(punctuations, '', result)
> 트윗에는 여러 문장 부호가 포함되어 있다. 이 역시 악성 트윗을 분류하는 데 영향이 없을 뿐만 아니라 글을 약간 지저분하게 만든다고 판단하여 정규표현식으로 지정하여 re.sub를 통해 제거하였다.

**8. 해시태그 처리**
- hashtag = '#(\w+)'
- result = re.sub(hashtag, '', result)
> 트윗에는 해시태그가 달려있기도 하다. 하지만 글의 내용과는 전혀 관련이 없는 해시태그들이 달려 있는 경우도 많아서, 이는 악성 트윗인지 아닌지를 분류하는 데 어려움을 준다고 판단하였다. 따라서 해시태그를 나타내는 정규표현식을 쓴 후 re.sub를 이용하여 제거하였다.

**9. 앞뒤 공백 제거**
- result = result.strip()
> 크게 중요하지는 않지만 앞뒤 공백을 제거하면 조금이라도 깔끔해질 것 같아서 하였다.

**10. 표제어 추출**
- 표제어 추출은 lemmatize(word) 함수를 따로 정의하여 이루어졌다.
- 이를 위해 from nltk.stem import WordNetLemmatizer를 한 뒤 nltk.download('wordnet')를 하여 필요한 것들을 불러왔다. 그 뒤 lemmatize(word) 함수에서 lemmatizer를 WordNetLemmatizer()로 설정한 뒤, return lemmatizer.lemmatize(word)를 통해 lemmatize(word) 함수가 lemmatize된 word를 반환하도록 만들었다.
> preprocess(text) 함수 마지막에 return lemmatize(result)를 함으로써 데이터 전처리를 할 때 lemmatize가 되게 하였다.

<br/>

그 뒤 마지막으로 corpus = data['tweet'].apply(preprocess)를 통해 data['tweet']의 트윗들에 preprocess를 적용하여, 데이터가 전처리된 하나의 corpus를 만들었다. corpus를 확인해보면 깔끔한 트윗들이 들어 있음을 알 수 있다.

<br/>

데이터 전처리를 할 때, stopwords를 제외할지 말지에 대한 선택을 해야 했다. 코드 맨 앞쪽에 주석처리된 5줄이 있는데, 이것이 그에 대한 부분이다. nltk.download('stopwords'), from nltk.corpus import stopwords를 통해 stopwords 설정을 위해 필요한 것들은 불러오고, stopwords = stopwords.words("english")를 통해 영어의 stopwords를 stopwords로 설정해준다. (여기서 stopwords란 불용어를 의미하는데, 이는 흔히 사용되는 그런 단어들을 의미한다.) 이렇게 만든 stopwords에 exclusions = ["#ff", "ff", "rt", ".", ",", '"', "'"]와 stopwords.extend(exclusions)를 통해 트윗에서 흔히 사용되는 문자열과 몇몇 문장 부호도 추가해준다. 이렇게 stopwords를 구성한 것이다.

하지만 stopwords를 주석처리하고 사용하지 않은 까닭은, stopwords를 데이터에서 제거하지 않았을 때의 결과가 stopwords를 제거하였을 때의 결과보다 좋았기 때문이다. 따라서 stopwords 코드는 주석처리하고 사용하지 않았다.

같은 이유로 preprocess(text) 함수에서 sentence = sentence.lower()를 주석처리하였다. 이 역시 sentence를 소문자화하지 않은 것이 소문자화한 것보다 결과가 좋았기 때문이다.

결론적으로 본 보고서에서는 위 10가지 항목들을 통해 트윗의 데이터들을 전처리하였다.




In [ ]:
import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# stopwords = stopwords.words("english")
# exclusions = ["#ff", "ff", "rt", ".", ",", '"', "'"]
# stopwords.extend(exclusions)

import re
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

def lemmatize(word):
  lemmatizer = WordNetLemmatizer()
  return lemmatizer.lemmatize(word)

def preprocess(sentence):
  # sentence = sentence.lower()
  space_pattern = '\s+'
  url = ('http\S+')
  rt_mention = '([Rr][Tt]\s@[A-Za-z0-9_]+)'
  middle_mention = '(@[A-Za-z0-9_]+)'
  not_words = "[^A-Za-z.,!?\'\"\s]*"
  punctuations = '[!\?.,:;\"\']'
  hashtag = '#(\w+)'
  sentence = sentence.replace('n\'t', ' not')
  sentence = sentence.replace('\'ll', ' will')
  sentence = sentence.replace('\'m', ' am')
  sentence = sentence.replace('\'re', ' are')
  result = re.sub(space_pattern, ' ', sentence)
  result = re.sub(url, '', result)
  result = re.sub(rt_mention, '', result)
  result = re.sub(middle_mention, '', result)
  result = re.sub(not_words, '', result)
  result = re.sub(punctuations, '', result)
  result = re.sub(hashtag, '', result)
  result = result.strip()
  return lemmatize(result)

corpus = data['tweet'].apply(preprocess)
corpus

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


0        As a woman you should not complain about clean...
1        boy dats coldtyga dwn bad for cuffin dat hoe i...
2        Dawg  You ever fuck a bitch and she start to c...
3                                   she look like a tranny
4        The shit you hear about me might be true or it...
                               ...                        
20437    Farra from teen mom has  sex tapes on pornhub ...
20438    Day  of FXXs Simpsons marathon  Barts Dog Gets...
20439      I almost ran this fine red bitch over Damn  lol
20440                                    they trash though
20441    Criticize Amanda for saying the n word sure bu...
Name: tweet, Length: 20000, dtype: object

## 특성(Features)

사용한 특성값은 크게 네 가지로, TF-IDF, 감정어휘, 단어의 개수, hate/offensive 단어의 개수가 있다. 

x1.  TF-IDF

*   ngram_range=(1,3): 유니그램+바이그램+트라이그램
> 다양한 조합을 시도해본 결과 (1,3)인 경우 가장 성능이 좋았다.
*   lowercase=False: 대소문자 구별 유지
> 전처리 시 모두 소문자화 하는 것보다 소문자화를 하지 않고, 대소문자 구별을 유지하는 경우에 더 성능이 좋았다.
*   decode_error=False: 디코딩 에러를 임의의 문자로 대체
*   use_idf=False: idf값을 사용하지 않음.
*   smooth_idf=False: idf의 분모에 더해지던 1을 idf 자체에 더함. 
*   norm=None: 각 피처 벡터를 정규화하지 않음.
> decode_error, use_idf, smooth_idf, norm 파라미터는 원저자의 코드를 참고하였다. 해당 파라미터를 넣은 경우 성능이 더 좋았기 때문에 결과적으로 코드에 포함하게 되었다. 

In [ ]:
# x1 TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,3), lowercase=False, decode_error='replace', 
                             use_idf=False, smooth_idf=False, norm=None)
x = vectorizer.fit_transform(corpus)

x2.   감정어휘
> 원저자의 코드를 참고하여 함수를 구성하였다. 소셜미디어 분석에 유용한 vader를 이용하여 감정지수를 도출했다. sentiment_analyzer.polarity_scores를 이용하여 감정지수를 모두 구한 후 종류별로 나누어 지수를 개별적으로 반환하는 함수들을 각각 만들어주었다.
*   sentiment_pos(text): 긍정 지수
*   sentiment_neg(text): 부정 지수
*   sentiment_neu(text): 중립 지수
*   sentiment_compound(text): 긍정, 부정, 중립을 종합한 종합적인 지수


In [ ]:
# x2 감정어휘
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

def sentiment_pos(text):
    sentiment = sentiment_analyzer.polarity_scores(text)
    return sentiment['pos']

def sentiment_neg(text):
    sentiment = sentiment_analyzer.polarity_scores(text)
    return sentiment['neg']

def sentiment_neu(text):
    sentiment = sentiment_analyzer.polarity_scores(text)
    return sentiment['neu']

def sentiment_compound(text):
    sentiment = sentiment_analyzer.polarity_scores(text)
    return sentiment['compound']

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "




x3.   단어의 개수
*   num_words(text): 트윗 단어의 개수
*   num_unique_terms(text): 중복을 제거한 트윗 단어의 개수



In [ ]:
# x3 단어 개수
def num_words(text):
  words = preprocess(text)
  num = len(words.split())
  return num

def num_unique_terms(text):
  words = preprocess(text)
  num_unique = len(set(words.split()))
  return num_unique



x4.   hate/offensive 단어의 개수

* count_hate_nouns(text): 혐오표현에 사용되는 단어가 얼마나 포함되는지를 세는 함수.  
hate_words의 목록은 Björn Technau의 'Going beyond hate speech: The pragmatics of ethnic slur terms' 기사를 참고하여 구성하였다.
* count_offensive_nouns(text): 공격적인 단어가 얼마나 포함되는지를 세는 함수.  
offensive_words의 목록은 임의로 구성하였다.



In [ ]:
# x4 hate/offensive 단어 개수
def count_hate_nouns(text):
  hate_words = ['nigger', 'niggers', 'nigguh', 'nigga', 'niggas', 'nigguhs', 
                'kike', 'kikes', 'kraut', 'krauts', 'faggot', 'faggots', 'spaz', 'spazzes']
  count = 0
  words = preprocess(text)
  for w in hate_words:
    if w in words:
      count += 1
  return count

def count_offensive_nouns(text):
  offensive_words = ['fuck', 'bitch', 'dick', 'shit', 'bitches']
  count = 0
  words = preprocess(text)
  for w in offensive_words:
    if w in words:
      count += 1
  return count

* extract_features(text): x2, x3, x4의 값을 구해 Series 형태로 반환하는 함수.

In [ ]:
def extract_features(text):
    x = []
    x.append(sentiment_pos(text))
    x.append(sentiment_neg(text))
    x.append(sentiment_neu(text))
    x.append(sentiment_compound(text))
    x.append(num_words(text))
    x.append(num_unique_terms(text))
    x.append(count_hate_nouns(text))
    x.append(count_offensive_nouns(text))
    return pd.Series(x)

* features: corpus의 x2, x3, x4 값을 구한 행렬

In [ ]:
from tqdm import tqdm
tqdm.pandas()
features = corpus.progress_apply(extract_features)

100%|██████████| 20000/20000 [00:26<00:00, 750.57it/s]


* concatenate 함수를 사용하면 tfidf 행렬이 sparse matrix이기 때문에 에러가 발생했다.  
따라서 scipy의 sparse.hstack 함수를 사용하여 x1과 x2, x3, x4를 합쳐주었다.

> 참고: scipy 홈페이지(https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.hstack.html)

In [ ]:
from scipy import sparse
x = sparse.hstack((x, features))

## 훈련집합, 개발집합 분리

특성값 x1, x2, x3, x4를 합친 값인 x와 클래스 값 y의 훈련집합과 개발집합을 분리하였다.  
random_state의 경우 모델을 실행할 때 같은 값이 나오도록 하기 위해 고정해주었다. 

In [ ]:
y = data['class'].copy()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_dev, y_train, y_dev = train_test_split(x, y, random_state=42, test_size=0.1)

## 분류기 선택 및 결과

### Logistic Regression

Logistc Regression을 분류기 모델로 선택하고,  
GridSearchCV를 이용하여 5-fold 교차 검증과 하이퍼 파라미터 튜닝을 동시에 진행했다.

+ hate speech, offensive speech, neither의 분포가 불균형하기 때문에 class_weight를 'balanced'로 설정했다.
+ 규제 강도를 의미하는 C, 규제에 사용된 기준을 지정하는 penalty는 하이퍼 파라미터 튜닝을 진행했다.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

logistic = LogisticRegression(random_state=10, verbose=1, class_weight='balanced', max_iter = 500)
log_params = {
    'C':[0.1, 1, 3, 5, 7],
    'penalty':["l2", "l1"]
}

grid_log = GridSearchCV(logistic, param_grid = log_params, cv = 5)

발생하는 warning을 무시해주기 위해 아래 코드를 넣어주었다.

In [ ]:
import warnings
warnings.filterwarnings(action = 'ignore')

In [ ]:
grid_log.fit(x_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight='balanced', max_iter=500,
                                          random_state=10, verbose=1),
             param_grid={'C': [0.1, 1, 3, 5, 7], 'penalty': ['l2', 'l1']})

튜닝 결과 규제 강도인 C가 7일 때, L2 규제를 이용할 때 score 0.8918로 가장 성능이 좋았다.

In [ ]:
print(grid_log.best_params_)
print(grid_log.best_score_)

{'C': 7, 'penalty': 'l2'}
0.8918888888888888


### 결과

test data를 바탕으로 실험해본 결과, 최종 accuracy_score는 0.89였다.   
각 label별 f1 score의 경우 원저자와 마찬가지로 hate speech의 경우가 0.42로 가장 낮았다.   
offensive speech, neither의 f1 score는 각각 0.93, 0.86으로 좋은 성능을 보였다.


결과를 정리하자면, 아래와 같다.
- accuracy_score (정확도) = 0.89
- precision (정밀도)
> 0번 범주 = 0.47  
1번 범주 = 0.94  
2번 범주 = 0.80
- recall (재현도)
> 0번 범주 = 0.38  
1번 범주 = 0.92  
2번 범주 = 0.92
- f1 score (f1 값)
> 0번 범주 = 0.42  
1번 범주 = 0.93  
2번 범주 = 0.86




In [ ]:
true = y_dev
pred = grid_log.predict(x_dev)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(true, pred))
print(confusion_matrix(true, pred))
print(classification_report(true, pred))

0.89
[[  46   57   17]
 [  50 1399   67]
 [   1   28  335]]
              precision    recall  f1-score   support

           0       0.47      0.38      0.42       120
           1       0.94      0.92      0.93      1516
           2       0.80      0.92      0.86       364

    accuracy                           0.89      2000
   macro avg       0.74      0.74      0.74      2000
weighted avg       0.89      0.89      0.89      2000

